# Library import 

In [1]:
import category_encoders as ce

In [2]:
import os
import random
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

from omegaconf import OmegaConf, DictConfig

from tqdm.auto import tqdm
import warnings
import wandb
from datetime import datetime
import re
from typing import Tuple

warnings.filterwarnings("ignore")

In [3]:
def sanitize_filename(filename):
    # Remove characters that are not allowed in Windows file names
    # (e.g., : / \ ? * < > | ")
    filename = re.sub(r'[\\/:*?"<>|]', '_', filename)
    return filename

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Configuration

In [5]:
project_root = os.getcwd()
data_root = os.path.join(project_root, "data")

In [6]:
cfg_dict: dict = {
    "WINDOW_SIZE" : 90,
    "PREDICT_SIZE" : 21,
    "EPOCHS" : 20,
    "LEARNING_RATE" : 1e-3,
    "BATCH_SIZE" : 1024,
    "NUM_WORKERS" : 0,
    "SEED" : 29,
    "input_size" : 5,
    "hidden_size" : 1024,
    "output_size" : 21,
    "num_layers" : 3,
    "num_attention_heads" : 4,
    "feedforward_dim" : 25,
    "dropout_rate" : 0.2,
    "hidden_sizes" : [512, 256, 128, 64]
}

cfg = OmegaConf.create(cfg_dict)
print(OmegaConf.to_yaml(cfg))

WINDOW_SIZE: 90
PREDICT_SIZE: 21
EPOCHS: 20
LEARNING_RATE: 0.001
BATCH_SIZE: 1024
NUM_WORKERS: 0
SEED: 29
input_size: 5
hidden_size: 1024
output_size: 21
num_layers: 3
num_attention_heads: 4
feedforward_dim: 25
dropout_rate: 0.2
hidden_sizes:
- 512
- 256
- 128
- 64



### SET SEED

In [7]:
random.seed(cfg["SEED"])
os.environ["PYTHONHASHSEED"] = str(cfg["SEED"])
np.random.seed(cfg["SEED"])
torch.manual_seed(cfg["SEED"])
torch.cuda.manual_seed(cfg["SEED"])
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = False # 실험시 False

# Data Load

In [8]:
train = pd.read_csv(data_root+"\\train.csv")
train.drop(["ID", "제품"], axis=1, inplace=True)

In [9]:
# 숫자형 변수들의 min-max scaling을 수행하는 코드입니다.
numeric_cols = train.columns[4:]
# 칵 column의 min 및 max 계산
min_values = train[numeric_cols].min(axis=1)
max_values = train[numeric_cols].max(axis=1)
# 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 1로 대체
ranges = max_values - min_values
ranges[ranges == 0] = 1
# min-max scaling 수행
train[numeric_cols] = (train[numeric_cols].subtract(min_values, axis=0)).div(ranges, axis=0)
# max와 min 값을 dictionary 형태로 저장
scale_min_dict = min_values.to_dict()
scale_max_dict = max_values.to_dict()

In [10]:
# 범주형 데이터를 바꾸기 위한 레이블 인코딩
# 원핫인코딩, 카테고리 인코딩등 여러 방법 고려 필요

categorical_col =  ["대분류", "중분류", "소분류", "브랜드"]
for col in categorical_col:
    encoder = ce.CountEncoder(cols=col, normalize=True)
    encoder.fit(train[col])
    train[col] = encoder.transform(train[col])

In [11]:
def make_train_data(data, train_size=cfg["WINDOW_SIZE"], predict_size=cfg["PREDICT_SIZE"]):
    '''
    학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성
    data : 일별 판매량
    train_size : 학습에 활용할 기간
    predict_size : 추론할 기간
    '''
    num_rows = len(data)
    window_size = train_size + predict_size
    
    input_data = np.empty((num_rows * (len(data.columns) - window_size + 1), train_size, len(data.iloc[0, :4]) + 1))
    target_data = np.empty((num_rows * (len(data.columns) - window_size + 1), predict_size))
    
    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, 4:])
        
        for j in range(len(sales_data) - window_size + 1):
            window = sales_data[j : j + window_size]
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
            input_data[i * (len(data.columns) - window_size + 1) + j] = temp_data
            target_data[i * (len(data.columns) - window_size + 1) + j] = window[train_size:]
    
    return input_data, target_data

In [12]:
def make_predict_data(data, train_size=cfg["WINDOW_SIZE"]):
    '''
    평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성
    data : 일별 판매량
    train_size : 추론을 위해 필요한 일별 판매량 기간 (= 학습에 활용할 기간)
    '''
    num_rows = len(data)
    
    input_data = np.empty((num_rows, train_size, len(data.iloc[0, :4]) + 1))
    
    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, -train_size:])
        
        window = sales_data[-train_size : ]
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
        input_data[i] = temp_data
    
    return input_data

In [13]:
train

,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0.689364,0.157395,0.027124,0.000063,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000
1,0.027942,0.025802,0.014726,0.000189,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.111111,0.333333,0.222222,0.00000,0.00000,0.222222,0.000000
2,0.027942,0.025802,0.014726,0.000189,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000
3,0.027942,0.025802,0.014726,0.000189,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000
4,0.237130,0.227061,0.089742,0.000755,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,0.027942,0.025802,0.003713,0.000378,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000
15886,0.027942,0.025802,0.014726,0.000378,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.024390,0.000000,0.016260,0.03252,0.00813,0.008130,0.024390
15887,0.027942,0.025802,0.014726,0.000378,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000
15888,0.027942,0.025802,0.014726,0.000378,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.142857


In [14]:
train_input, train_target = make_train_data(train)
test_input = make_predict_data(train)

  0%|          | 0/15890 [00:00<?, ?it/s]

  0%|          | 0/15890 [00:00<?, ?it/s]

In [15]:
train_target.shape

(5609170, 21)

In [16]:
train

,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0.689364,0.157395,0.027124,0.000063,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000
1,0.027942,0.025802,0.014726,0.000189,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.111111,0.333333,0.222222,0.00000,0.00000,0.222222,0.000000
2,0.027942,0.025802,0.014726,0.000189,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000
3,0.027942,0.025802,0.014726,0.000189,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000
4,0.237130,0.227061,0.089742,0.000755,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,0.027942,0.025802,0.003713,0.000378,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000
15886,0.027942,0.025802,0.014726,0.000378,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.024390,0.000000,0.016260,0.03252,0.00813,0.008130,0.024390
15887,0.027942,0.025802,0.014726,0.000378,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000
15888,0.027942,0.025802,0.014726,0.000378,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.142857


In [17]:
# Train / Validation Split
data_len = len(train_input)
val_input = train_input[-int(data_len*0.2):]
val_target = train_target[-int(data_len*0.2):]
train_input = train_input[:-int(data_len*0.2)]
train_target = train_target[:-int(data_len*0.2)]

In [18]:
train_input.shape, train_target.shape, val_input.shape, val_target.shape, test_input.shape

((4487336, 90, 5),
 (4487336, 21),
 (1121834, 90, 5),
 (1121834, 21),
 (15890, 90, 5))

# DataSet

In [19]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])
    
    def __len__(self):
        return len(self.X)

In [20]:
train_dataset = CustomDataset(train_input, train_target)
train_dataloader = DataLoader(train_dataset, batch_size = cfg['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_input, val_target)
val_dataloader = DataLoader(val_dataset, batch_size = cfg['BATCH_SIZE'], shuffle=False, num_workers=0)

In [21]:
for sample in train_dataloader:
    print(sample[0].shape)
    print(sample[1].shape)
    break

torch.Size([1024, 90, 5])
torch.Size([1024, 21])


# Define Model

- 모델 구조 : Sequence to Sequence 

    - 인코더와 디코더로 이루어짐
    - LSTM (인코더) -> 시퀀스 전달
    - LSTM (디코더) -> 시퀀스를 받아 output 생성

In [22]:
class Encoder(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, num_layers: int):

        '''
        parameter
            - input_size : 입력 차원의 크기 # 현재 데이터셋의 경우 [batch_size, window_size(src_len), input_size] 로 구성
            - hidden_size : LSTM 모델의 히든 차원의 크기 -> 하이퍼 파라미터로 실험 필요
            - num_layer : LSTM 에서 층을 몇개를 쌓을지 -> 하이퍼 파라미터로 실험 필요
        '''
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.3,
            bidirectional=False,
        )

    def forward(self, x):
        # hidden state와 cell state를 초기화 (0행렬)을 통해 넘겨주어야 함
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.lstm(x, (h0, c0))
        
        return out

class Decoder(nn.Module):
    def __init__(self, hidden_size: int, output_size: int, num_layers: int):
        '''
        parameter
            - hidden_size : 모델의 hidden_size -> 하이퍼파라미터 변경 필요
            - output_size : 현재 task에서는 21일을 예측해야 하기 때문에 상수
            - num_layer : 모델에서 층을 몇층을 쌓을 지 -> 하이퍼파라미터 변경 필요
        '''
        super().__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size

        self.lstm = nn.LSTM(
            input_size=hidden_size,
            hidden_size=hidden_size // 2,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.3,
            bidirectional=False,
        )

        self.relu = nn.ReLU()
        # decoder의 경우 output을 출력해야 하기때문에 Linear층과 연결을 통해 output을 출력한다.
        # 현재 Sequential을 사용하여 기본 구조인 Linear -> Dropout -> Linear를 구성하였지만 이또한 변경 가능
        # output_size만 고정값이면 문제 없음
        self.fc = nn.Sequential(
            nn.Linear(self.hidden_size // 2, self.hidden_size//4),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(self.hidden_size // 4, self.output_size)
        )
        self.actv = nn.ReLU()

    def forward(self, x):
        # LSTM의 입력에 들어가는 hidden state와 cell state를 초기화
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size // 2).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size // 2).to(device)

        out, _ = self.lstm(x, (h0, c0))
        # tensor size를 맞추기 위하여 가장 마지막 output만 가져오기 위해서 [:, -1, :] 사용
        out = out[:, -1, :]
        out = self.actv(self.fc(out))
        out = out.squeeze(1)

        return out


class Seq2Seq(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, output_size: int, num_layers: int):
        super(Seq2Seq, self).__init__()
        self.encoder = Encoder(input_size, hidden_size, num_layers)
        self.decoder = Decoder(hidden_size, output_size, num_layers)

    def forward(self, x):
        encoder_out = self.encoder(x)
        decoder_out = self.decoder(encoder_out)
        return decoder_out

In [23]:
model = Seq2Seq(input_size=cfg["input_size"], output_size=cfg["output_size"], hidden_size=cfg["hidden_size"], num_layers=4)

In [24]:
print(model)

Seq2Seq(
  (encoder): Encoder(
    (lstm): LSTM(5, 1024, num_layers=4, batch_first=True, dropout=0.3)
  )
  (decoder): Decoder(
    (lstm): LSTM(1024, 512, num_layers=4, batch_first=True, dropout=0.3)
    (relu): ReLU()
    (fc): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.5, inplace=False)
      (3): Linear(in_features=256, out_features=21, bias=True)
    )
    (actv): ReLU()
  )
)


# model compile

In [25]:
# Warmup Scheduler
class WarmupLR(optim.lr_scheduler.LambdaLR):

    def __init__(
        self,
        optimizer: optim.Optimizer,
        warmup_end_steps: int,
        last_epoch: int = -1,
    ):
        
        def wramup_fn(step: int):
            if step < warmup_end_steps:
                return float(step) / float(max(warmup_end_steps, 1))
            return 1.0
        
        super().__init__(optimizer, wramup_fn, last_epoch)


In [26]:
import torch.optim.lr_scheduler as lr_scheduler

In [27]:
# set up gpu
gpu = 0

# define model
if gpu is not None:
    model.cuda(gpu)
model_name = type(model).__name__

# define loss
loss_function = nn.MSELoss()
scheduler = None
# define optimizer
lr = cfg["LEARNING_RATE"]
optimizer = optim.Adam(model.parameters(), lr=lr)
# 기존 Adam에 Weigth decay를 적용한 옵티마이저로 더 안정적인 학습이 가능
# optimizer = optim.AdamW(model.parameters(), lr=lr)
optimizer_name = type(optimizer).__name__

# define scheduler
# 사용하지 않는 스케줄러는 주석 처리
# warmup 스케줄러 - 초기에는 작은 값으로 학습하다 학습이 안정화되면 초기 학습률로 전환하는 방법
# scheduler = WarmupLR(optimizer, 1500)
# StepLR 스케줄러  - 일정 스텝마다 학습률에 감마값을 곱하여 학습률을 조정
# step_size = 10
# gamma = 0.5
# scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

# # ExponentialLR 스케줄러 - 학습률의 곡선이 지수 함수 형태를 만들어 줌
# exponential_gamma = 0.95
# scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=exponential_gamma)

# # CosineAnnealingLR 스케줄러 - 코사인 그래프를 그리면서 학습률이 진동하는 방식, 단순히 감소가 아닌 진동하며 최적점을 찾아감
# scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)  # T_max는 주기의 반복 횟수

# # ReduceLROnPlateau 스케줄러 - 몇번 이상(patience)가 감소하지 않으면 학습률을 factor만큼 감소시킴 
# reduce_lr_patience = 5
# reduce_lr_factor = 0.1
# reduce_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=reduce_lr_patience, factor=reduce_lr_factor)

scheduler_name = type(scheduler).__name__ if scheduler is not None else "no"

# define wandb
# project_name = "LG_AIMERS_Sales_Forecast"
# current_time = datetime.now().replace(microsecond=0).isoformat().replace(":", "_")
# run_name = f"{current_time}_{model_name}_{optimizer_name}_optim_{lr}_with_{scheduler_name}"
# run_name = sanitize_filename(run_name)
# run_tags = [project_name]
# wandb.init(
#     project=project_name,
#     name=run_name,
#     tags=run_tags,
#     config={"lr": lr, "model_name": model_name, "optimizer_name": optimizer_name, "scheduler_name": scheduler_name},
#     reinit=True
# )
# wandb.watch(model)

# Train

In [28]:
clip_value = 1.0

In [29]:
def train(model, optimizer, train_dataloader, val_dataloader, device, patience=5):
    
    model.to(device)
    criterion = nn.MSELoss().to(device)
    # 초기 로스를 무한으로 설정
    best_loss = np.inf
    best_model = None
    # Early Stopping Counter
    counter = 0
    # Early Stopping Patience
    patience = 5
    best_model_state_dict = None
    
    for epoch in range(1, cfg['EPOCHS']+1):
        model.train()
        train_loss = []
        for X, Y in tqdm(iter(train_dataloader)):
            X = X.to(device)
            Y = Y.to(device)
            
            # Foward
            optimizer.zero_grad()

            # get prediction
            output = model(X)
            
            loss = criterion(output, Y)
            
            # back propagation
            loss.backward()
            # Apply gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            optimizer.step()
            # Perform LR scheduler Work
            if scheduler is not None:
                scheduler.step()
            
            train_loss.append(loss.item())
        
        val_loss = validation(model, val_dataloader, criterion, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')
        
        if best_loss > val_loss:
            best_loss = val_loss
            # 로스가 감소하였을 때 모델 갱신 및 저장
            best_model_state_dict = model.state_dict()
            torch.save(best_model_state_dict, "best_model.pth")
            counter = 0
            print('Model Saved')
        else:
            counter += 1
            print(f" Early Stopping count : {counter}")
            if counter >= patience:
                print("Early stopping.")
                break
        
        # # WandB logging
        # wandb.log({
        #     "Epoch": epoch,
        #     "Train Loss": np.mean(train_loss),
        #     "Validation Loss": val_loss,
        # })
        

    model.load_state_dict(best_model_state_dict)
    return model


def validation(model, val_dataloader, criterion, device):
    model.eval()
    val_loss = []
    
    with torch.no_grad():
        for X, Y in tqdm(iter(val_dataloader)):
            X = X.to(device)
            Y = Y.to(device)
            
            output = model(X)
            loss = criterion(output, Y)
            
            val_loss.append(loss.item())
    return np.mean(val_loss)

In [30]:
infer_model = train(model, optimizer, train_dataloader, val_dataloader, device)

  0%|          | 0/4383 [00:00<?, ?it/s]

  0%|          | 0/1096 [00:00<?, ?it/s]

Epoch : [1] Train Loss : [0.03077] Val Loss : [0.03019]
Model Saved


  0%|          | 0/4383 [00:00<?, ?it/s]

  0%|          | 0/1096 [00:00<?, ?it/s]

Epoch : [2] Train Loss : [0.03072] Val Loss : [0.03021]
 Early Stopping count : 1


  0%|          | 0/4383 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [30]:
test_dataset = CustomDataset(test_input, None)
test_dataloader = DataLoader(test_dataset, batch_size = cfg['BATCH_SIZE'], shuffle=False, num_workers=cfg["NUM_WORKERS"])

In [31]:
loaded_model = Seq2Seq(input_size=cfg["input_size"], hidden_size=cfg["hidden_size"], output_size=cfg["output_size"], num_layers=4)
state = torch.load("best_model.pth")
loaded_model.load_state_dict(state)

<All keys matched successfully>

In [35]:
loaded_model.to(device)

Seq2Seq(
  (encoder): Encoder(
    (lstm): LSTM(5, 512, num_layers=4, batch_first=True, dropout=0.3)
  )
  (decoder): Decoder(
    (lstm): LSTM(512, 256, num_layers=4, batch_first=True, dropout=0.3)
    (relu): ReLU()
    (fc): Sequential(
      (0): Linear(in_features=256, out_features=128, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.5, inplace=False)
      (3): Linear(in_features=128, out_features=21, bias=True)
    )
    (actv): ReLU()
  )
)

In [36]:
def inference(model, test_loader, device):
    predictions = []
    
    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            X = X.to(device)
            
            output = model(X)
            
            output = output.cpu().numpy()
            
            predictions.extend(output)
    
    return np.array(predictions)

In [37]:
pred = inference(loaded_model, test_dataloader, device)

  0%|          | 0/16 [00:00<?, ?it/s]

In [38]:
# 추론 결과를 inverse scaling
for idx in range(len(pred)):
    pred[idx, :] = pred[idx, :] * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]
    
# 결과 후처리
pred = np.round(pred, 0).astype(int)

In [39]:
pred

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 2, 2, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [3, 2, 2, ..., 2, 1, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [40]:
pred.shape

(15890, 21)

# Submission

In [41]:
submit = pd.read_csv(data_root + "/sample_submission.csv")
submit.iloc[:,1:] = pred
submit.head()

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,2,2,1,1,1,1,2,2,...,1,1,1,2,2,2,1,1,1,1
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,1
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
4,4,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,2,2,2,2,2


In [42]:
submit.to_csv('./s2s2.csv', index=False)